# 문자를 숫자로 표현하는 방법

In [4]:
S1='나는 책상 위에 사과를 먹었다'
S2='알고 보니 그 사과는 Jason 것이었다'
S3='그래서 Jason에게 사과를 했다'

# Tokenization : 문장을 의미 있는 부분으로 나누는 과정 -> 띄어쓰기 이용
print(S1.split())
print(S2.split())
print(S3.split())

['나는', '책상', '위에', '사과를', '먹었다']
['알고', '보니', '그', '사과는', 'Jason', '것이었다']
['그래서', 'Jason에게', '사과를', '했다']


In [5]:
# 겹치는 Token을 따로 모을 필요는 없기 때문에 각 Token에 Index를 지정해 사전 형식으로 모으기
token2idx={}
index=0

for sentence in [S1, S2, S3]:
  tokens=sentence.split()
  for token in tokens:
    if token2idx.get(token)==None: # get(x) 함수는 x라는 Key에 대응되는 Value를 돌려준다
      token2idx[token]=index
      index+=1

print(token2idx)

{'나는': 0, '책상': 1, '위에': 2, '사과를': 3, '먹었다': 4, '알고': 5, '보니': 6, '그': 7, '사과는': 8, 'Jason': 9, '것이었다': 10, '그래서': 11, 'Jason에게': 12, '했다': 13}


위와 같이 Token을 저장해 놓은 사전인 token2idx를 Vocabulary라 하며, Token의 저장과 관리뿐 아니라 저장할 때 함께 저장한 Index를 이용해 문자를 숫자로 바꾸는 데 사용

In [6]:
# Token의 Index 해당 숫자로 각 문장을 바꿔보기
def indexed_sentence(sentence):
  return[token2idx[token] for token in sentence]

print(indexed_sentence(S1.split()))
print(indexed_sentence(S2.split()))
print(indexed_sentence(S3.split()))

[0, 1, 2, 3, 4]
[5, 6, 7, 8, 9, 10]
[11, 12, 3, 13]


## 2.1 Corpus & Out-of-Vocabulary

OOV(Out-of-Vocabulary) : Token을 저장해둔 Vocabulary에 Token이 없어서 처음 본 Token이 나오는 현상, 이런 상황을 대비해 특수한 Token인 <unk> Token을 만들어 Vocabulary에 없는 Token이 나올 경우 변환

In [8]:
# 기존 token 사전에 <unk> token 추가
token2idx={t:i+1 for t, i in token2idx.items()} # 0을 만들기 위해 value에 1씩 추가
token2idx['<unk>']=0

# token이 없을 경우 <unk> token의 0을 반환
def indexed_sentence_unk(sentence):
  return [token2idx.get(token, token2idx['<unk>']) for token in sentence]

S4='나는 책상 위에 배를 먹었다'
indexed_sentence_unk(S4.split())

[1, 2, 3, 0, 5]

But 애초에 사전을 풍부하게 만들면 OOV 문제가 해결

말뭉치(Corpus) : Token을 모으기 위해 모아 놓은 문장의 모음

## 2.2 Byte Pair Encoding(BPE)

다른 Tokenization 방법

In [9]:
# Character based tokenization
idx2char={0:'<pad>', 1:'<unk>'}

srt_idx=len(idx2char) # 기존 dict에서 숫자 추가
for x in range(32, 127): # 특수문자 및 숫자, 영어
  idx2char.update({srt_idx : chr(x)})
  srt_idx+=1


# 한글 추가
for x in range(int('0x3131', 16), int('0x3163', 16)+1): # 모든 완성형 한글
  idx2char.update({srt_idx : chr(x)})
  srt_idx+=1

for x in range(int('0xAC00', 16), int('0xD7A3', 16)+1): # 자모
  idx2char.update({srt_idx : chr(x)})
  srt_idx+=1

char2idx={v:k for k,v in idx2char.items()}
print([char2idx.get(c,0) for c in '그래서 Jason에게 사과를 했다'])
print([char2idx.get(c,0) for c in 'ㅇㅋ! ㄱㅅㄱㅅ'])

[652, 3116, 5552, 2, 44, 67, 85, 81, 80, 6756, 288, 2, 5440, 400, 3600, 2, 10780, 1912]
[119, 123, 3, 2, 97, 117, 97, 117]


기존에 있었던 문장은 물론, 줄임말이나 신조어에 대한 OOV 걱정 X & 사전의 사이즈도 크지 않다. But 글자 하나는 보통 특정 의미를 갖고 있지 않아 모델이 글자의 조합에 대한 정보나 의미를 담도록 설계하는 것이 쉽지 않다.

In [11]:
# n-gram Tokenization : 글자보다는 좀 더 긴 형태의 Token을 만들어내기 위해 사용하는 방법, 띄어쓰기 포함

S1='나는 책상 위에 사과를  먹었다'
print([S1[i:i+1] for i in range(len(S1))]) # uni-gram : character based tokenization
print([S1[i:i+2] for i in range(len(S1))]) # bi-gram : 2개의 글자를 Token 단위로
print([S1[i:i+3] for i in range(len(S1))]) # tri-gram : 3개의 글자를 Token 단위로

['나', '는', ' ', '책', '상', ' ', '위', '에', ' ', '사', '과', '를', ' ', ' ', '먹', '었', '다']
['나는', '는 ', ' 책', '책상', '상 ', ' 위', '위에', '에 ', ' 사', '사과', '과를', '를 ', '  ', ' 먹', '먹었', '었다', '다']
['나는 ', '는 책', ' 책상', '책상 ', '상 위', ' 위에', '위에 ', '에 사', ' 사과', '사과를', '과를 ', '를  ', '  먹', ' 먹었', '먹었다', '었다', '다']


띄어쓰기 기준의 Token에 대해 n-gram을 사용할 경우, 연속적으로 사용되는 용어를 잘 찾아낼 수 있다. But 쓸모 없는 조합이 너무 많이 생성 

In [12]:
# BPE(Byte Pair Encoding) : 반복적으로 나오는 데이터의 연속된 패턴을 치환하는 방식
import re, collections

def get_stats(vocab): 
    pairs = collections.defaultdict(int) # 딕셔너리(dictionary)와 거의 비슷하지만 key값이 없을 경우 미리 지정해 놓은 초기(default)값을 반환하는 dictionary
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>' : 5, 'l o w e r </w>' : 2, 'n e w e s t </w>':6, 'w i d e s t </w>':3} # 단어 횟수를 기록한 사전, 이 때 사전의 단어 글자는 모두 띄어 표현
num_merges = 10 # 미리 정해 놓은 횟수만큼 과정 반복

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get) # 각 단어에 대해 연속된 2개의 글자의 숫자를 세어 가장 많이 나오는 글자 2개의 조합을 찾기
    vocab = merge_vocab(best, vocab) # 두 글자를 합쳐 기존의 사전의 단어를 수정
    print(f'Step {i + 1}')
    print(best)
    print(vocab)
    print('\n')

Step 1
('e', 's')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


Step 2
('es', 't')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


Step 3
('est', '</w>')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 4
('l', 'o')
{'lo w </w>': 5, 'lo w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 5
('lo', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}


Step 6
('n', 'e')
{'low </w>': 5, 'low e r </w>': 2, 'ne w est</w>': 6, 'w i d est</w>': 3}


Step 7
('ne', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'new est</w>': 6, 'w i d est</w>': 3}


Step 8
('new', 'est</w>')
{'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


Step 9
('low', '</w>')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


Step 10
('w', 'i')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}




"est", "low" 등 자주 등장하는 글자의 연속인 subwords를 찾을 수 있음.

In [13]:
# 이전의 띄어쓰기나 Character n-gram Tokenization 결과와 비교

S1='나는 책상 위에 사과를 먹었다'
S2='알고 보니 그 사과는 Jason 것이었다'
S3='그래서 Jason에게 사과를 했다'

token_counts={} # 언급된 token의 개수 세기
index=0

for sentence in [S1, S2, S3]:
  tokens=sentence.split()
  for token in tokens:
    if token_counts.get(token)==None:
      token_counts[token]=1
    else:
      token_counts[token]+=1

token_counts={" ".join(token) : counts for token, counts in token_counts.items()}
print(token_counts)

{'나 는': 1, '책 상': 1, '위 에': 1, '사 과 를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사 과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


In [14]:
num_merges=10

for i in range(num_merges):
  pairs=get_stats(token_counts)
  best=max(pairs, key=pairs.get)
  token_counts=merge_vocab(best, token_counts)
  print(f'Step {i + 1}')
  print(best)
  print(token_counts)
  print('\n')

Step 1
('사', '과')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과 를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 2
('사과', '를')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 3
('었', '다')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


Step 4
('J', 'a')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'Ja s o n': 1, '것 이 었다': 1, '그 래 서': 1, 'Ja s o n 에 게': 1, '했 다': 1}


Step 5
('Ja', 's')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'Jas o n': 1, '것 이 었다': 1, '그 래 서': 1, 'Jas o n 에 게': 1, '했 다': 1}


Step 6
('Jas', 'o')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 

## 2.3 Word Enbedding

문자를 숫자로 표현하는 방법

In [19]:
# 원-핫 인코딩 
S1='나는 책상 위에 사과를 먹었다'
S2='알고 보니 그 사과는 Jason 것이었다'
S3='그래서 Jason에게 사과를 했다'

token2idx={}
index=0

for sentence in [S1, S2, S3]:
  tokens = sentence.split()
  for token in tokens:
    if token2idx.get(token) == None:
      token2idx[token] = index
      index += 1

print(token2idx)

{'나는': 0, '책상': 1, '위에': 2, '사과를': 3, '먹었다': 4, '알고': 5, '보니': 6, '그': 7, '사과는': 8, 'Jason': 9, '것이었다': 10, '그래서': 11, 'Jason에게': 12, '했다': 13}


In [20]:
V = len(token2idx) # 단어 사전의 크기
token2vec = [([0 if i != idx else 1 for i in range(V)], idx, token) for token, idx in token2idx.items() ] # 각 Token은 그에 해당하는 Index의 값만 1의 값을 가진 벡터로 표현

for x in token2vec: # x에 vector & index 포함
  print("\t".join([str(y) for y in x]))

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	0	나는
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	1	책상
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	2	위에
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]	3	사과를
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]	4	먹었다
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]	5	알고
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]	6	보니
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]	7	그
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]	8	사과는
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]	9	Jason
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]	10	것이었다
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]	11	그래서
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]	12	Jason에게
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]	13	했다


In [22]:
# python numpy를 이용해 문장을 원-핫 인코딩으로 바꾸는 방법
import numpy as np

for sentence in [S1, S2, S3]:
  onehot_s=[]
  tokens=sentence.split()
  for token in tokens:
    if token2idx.get(token)!=None:
      vector=np.zeros((1,V))
      vector[:,token2idx[token]]=1
      onehot_s.append(vector)
    else:
      print("UNK")

  print(f"{sentence} : ")        
  print(np.concatenate(onehot_s, axis = 0))
  print('\n')

나는 책상 위에 사과를 먹었다 : 
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


알고 보니 그 사과는 Jason 것이었다 : 
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


그래서 Jason에게 사과를 했다 : 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]




## 예제 5-3 데이터 전처리 및 Pre-Trained Embedding Vector를 이용한 Vocabulary 생성하기

In [2]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

# Data Setting : Field 설정을 통해 여러 가지 작업을 미리 할 수 있음
TEXT=data.Field(batch_first=True, # batch size를 data shape axis의 가장 앞으로 설정하는 옵션
                fix_length=500, # sentence의 길이를 미리 제한하는 욥션
                tokenize=str.split, # tokenize를 설정하는 옵션 기본값은 띄어쓰기 기반의 파이썬의 string.split 함수
                pad_first=True, # fix_length 대비 짧은 문장의 경우 padding을 해야 하는데 padding을 앞에서 줄 것인지에 대한 옵션
                pad_token='[PAD]', # 위에서 설정한 padding에 대한 특수 token 설정
                unk_token='[UNK]') # token dictionary에 없는 token이 나왔을 경우 해당 token을 표현하는 특수 token

LABEL=data.LabelField(dtype=torch.float) # 가져올 데이터에 대한 type 설정 옵션
train_data, test_data=datasets.IMDB.splits(text_field=TEXT, label_field=LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 44.0MB/s]


In [3]:
# Data length
print(f'Train Data Length : {len(train_data.examples)}') # data.example : 데이터의 개수 확인
print(f'Test Data Length : {len(test_data.examples)}')

Train Data Length : 25000
Test Data Length : 25000


In [4]:
# Data Fields
print(train_data.fields)

{'text': <torchtext.legacy.data.field.Field object at 0x7fba067df210>, 'label': <torchtext.legacy.data.field.LabelField object at 0x7fba5ffa0f10>}


In [5]:
# Data Sample
print('---- Data Sample ----')
print('Input : ')
print(' '.join(vars(train_data.examples[1])['text']),'\n') # vars() 함수 이용 : 데이터의 값을 직접 확인 가능
print('Label : ')
print(vars(train_data.examples[1])['label'])

---- Data Sample ----
Input : 
This movie is best described or compare to "Big Fish" (the movie by Tim Burton). But it's a less glamorous and more in you face tale. And of course here it's not the father, but his grandfather who tells the stories.<br /><br />The movie's narrative also moves back and forth (so the story outline here at IMDb, might tell you more than you would like to read, before watching the movie). It's funny and engaging enough, even though you get from one story to another and have some dramatic moments too. It also surprises you here and there, with things you wouldn't expect. A nice little movie then, that deserves your attention, especially if you like movies like that! :o) 

Label : 
pos


In [6]:
# Data Cleansing : Text Data는 Tokenize를 하기 전에 Data를 살펴보고 Cleansing 작업을 해야 한다.
import re
def PreProcessingText(input_sentence):
  input_sentence=input_sentence.lower() # 소문자화
  input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
  input_sentence = re.sub('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
  input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
  if input_sentence:
    return input_sentence

for example in train_data.examples:
  vars(example)['text']=PreProcessingText(' '.join(vars(example)['text'])).split()
for example in test_data.examples:
  vars(example)['text']=PreProcessingText(' '.join(vars(example)['text'])).split()

In [9]:
# 주어진 data를 이용해 token vocabulary를 만드는 과정
model_config={'emb_type' : 'glove', 'emb_dim' : 300}

TEXT.build_vocab(train_data, 
                 min_freq=2, # vocab에 해당하는 token에 최소한으로 등장하는 횟수에 제한
                 max_size=None, # 전체 vocab size 자체에 제한
                 vectors="glove.6B.300d") # pre-trained vector를 가져와 vocab에 세팅하는 옵션. 원하는 embedding을 정해 string 형태로 설정하면 됨.

LABEL.build_vocab(train_data)
model_config['vocab_size']=len(TEXT.vocab)

In [8]:
# Vocabulary Info
print(f'Vocab Size : {len(TEXT.vocab)}')

print('Vocab Examples : ')
for idx, (k, v) in enumerate(TEXT.vocab.stoi.items()):
  if idx >= 10:
    break    
  print('\t', k, v)

print('---------------------------------')

# Label Info
print(f'Label Size : {len(LABEL.vocab)}')

print('Lable Examples : ')
for idx, (k, v) in enumerate(LABEL.vocab.stoi.items()):
  print('\t', k, v)

Vocab Size : 51956
Vocab Examples : 
	 [UNK] 0
	 [PAD] 1
	 the 2
	 and 3
	 a 4
	 of 5
	 to 6
	 is 7
	 in 8
	 it 9
---------------------------------
Label Size : 2
Lable Examples : 
	 neg 0
	 pos 1


In [10]:
# check embedding vectors
TEXT.vocab.vectors.shape

torch.Size([51956, 300])

In [11]:
import random

# spliting valid set
train_data, valid_data=train_data.split(random_state=random.seed(0), split_ratio=0.8)

In [13]:
model_config['batch_size']=30

train_iterator, valid_iterator, test_iterator=data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=model_config['batch_size'])

In [21]:
# 이 데이터로 모델의 feed-forward가 잘되는지 확인
import torch.nn as nn
import torch.nn.functional as F

class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(SentenceClassification, self).__init__()

        if model_config['emb_type'] == 'glove' or 'fasttext':
            self.emb = nn.Embedding(model_config['vocab_size'],
                                    model_config['emb_dim'],
                                    _weight = TEXT.vocab.vectors)
        else:
            self.emb = nn.Embedding(model_config['vocab_size'],
                                    model_config['emb_dim'])
        
        self.bidirectional = model_config['bidirectional']
        self.num_direction = 2 if model_config['bidirectional'] else 1
        self.model_type = model_config['model_type'] 

        self.RNN = nn.RNN (input_size = model_config['emb_dim'], # 입력받을 data의 크기 : embedding dimension을 설정
                           hidden_size = model_config['hidden_dim'],
                           dropout=model_config['dropout'], # dropout의 확률
                           bidirectional = model_config['bidirectional'], # 양방향 모델을 사용할 경우에 설정
                           batch_first = model_config['batch_first']) # data의 제일 처음 axis에 batch_size가 오도록 설정
        
        self.LSTM= nn.LSTM(input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout=model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.GRU = nn.GRU (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout=model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
    
        self.fc = nn.Linear(model_config['hidden_dim'] * self.num_direction, # 분류 문제 : class에 대한 score를 생성하기 위해 FC layer 1개 생성
                            model_config['output_dim'])
        
        self.drop = nn.Dropout(model_config['dropout'])

    def forward(self, x):
        
        emb = self.emb(x) # emb : (Batch_Size, Max_Seq_Length, Emb_dim)

        if self.model_type == 'RNN':
            output, hidden = self.RNN(emb) # output : (Batch_Size, Max_Seq_Length, Hidden_dim * num_direction), hidden : (num_direction, Batch_Size, Hidden_dim)
        elif self.model_type == 'LSTM':
            output, (hidden, cell) = self.LSTM(emb)
        elif self.model_type == 'GRU':
            output, hidden = self.GRU(emb)
        else:
            raise NameError('Select model_type in [RNN, LSTM, GRU]')

        last_output = output[:,-1,:] # last_output : (Batch_Size, Hidden_dim * num_direction)

        return self.fc(self.drop(last_output))

In [50]:
import sys

def train(model, iterator, optimizer, loss_fn, idx_Epoch, **model_params):
  Epoch_loss=0
  Eplch_acc=0

  model.train()
  batch_size=model_params['batch_size']

  for idx, batch in enumerate(iterator):
    optimizer.zero_grad(iterator) # initializing

    # forward
    predictions=model(batch.text).squeeze()
    loss=loss_fn(predictions, batch.label)
    acc=binary_accuracy(predictions, batch.label)

    sys.stdout.write(
                    "\r" + f"[Train] Epoch : {idx_epoch:^3}"\
                    f"[{(idx + 1) * batch_size} / {len(iterator) * batch_size} ({100. * (idx + 1) / len(iterator) :.4}%)]"\
                    f"  Loss: {loss.item():.4}"\
                    f"  Acc : {acc.item():.4}"\
                    )
    
    # backward
    loss.backward()
    optimizer.step()

    # update epoch performance
    Epoch_loss+=loss.item()
    Epoch_acc+=acc.item()

  return Epoch_loss/len(iterator), Epoch_acc/len(iterator)

In [24]:
def evaluate(model, iterator, loss_fn):
  Epoch_loss=0
  Epoch_acc=0

  # evaluation mode
  model.eval()
  with torch.no_grad():
    for batch in iterator:
      predictions=model(batch.text).sqeeze(1)
      loss=loss_fn(predictions, batch.label)
      acc=binary_accuracy(predictions, batch.label)

      Epoch_loss+=loss.item()
      Epoch_acc+=acc.item()

  return Epoch_loss/len(iterator), Epoch_acc/len(iterator)

In [25]:
model_config.update(dict(batch_first=True, model_type='RNN', bidirectional=True,
                         hidden_dim=128, output_dim=1, dropout=0)) # binary classification : output_dim=1
model_config['model_type']='RNN'

model=SentenceClassification(**model_config)
optimizer=torch.optim.Adam(model.parameters())
loss_fn=nn.BCEWithLogitsLoss() # BCEWithLogitsLoss : 내부에서 시그모이드를 적용하기 때문에 Sigmoid Layer를 통과시키지 않은 값을 Input으로 사용

In [26]:
N_EPOCH=5

best_valid_loss=float('inf')
model_name = f"{'bi-' if model_config['bidirectional'] else ''}{model_config['model_type']}_{model_config['emb_type']}"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn)
    print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt') # Early stopping을 위해 Validation Loss가 최저일 때의 모델을 저장해두고, 이를 나중에 불러와 Test Set에서 적용해 성능을 측정
        print(f'\t Saved at {epoch}-epoch')

    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')

---------------------------------
Model name : bi-RNN_glove
---------------------------------


KeyError: ignored

## 예제 5.5 Pre-Trained BERT Model을 이용한 모델 만들기

In [29]:
!pip install transformers

In [30]:
import re
import sys
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchtext.legacy import data
from torchtext.legacy import datasets

from transformers import BertTokenizer, BertModel

In [32]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(len(tokenizer.vocab))

max_input_length=tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

def new_tokenizer(sentence):
  tokens=tokenizer.tokenize(sentence)
  tokens=tokens[:max_input_length-2] # 문장의 앞에는 [CLS], 문장의 맨 뒤에는 [SEP] Token을 추가해야 하기 때문에 2개의 special token이 들어갈 자리 2개 미리 빼둠
  return tokens


30522
512


In [33]:
# 기본적인 전처리 & token을 index로 바꿔주는 과정
def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower() # 소문자화
    input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
    input_sentence = re.sub('[!"$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
    input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence

def PreProc(list_sentence): # BERT가 이미 갖고 있는 vocab를 사용해야 하기 때문에 전처리 과정에서 벡터 변환
    return [tokenizer.convert_tokens_to_ids(PreProcessingText(x)) for x in list_sentence]

In [34]:
TEXT=data.Field(batch_first=True, use_vocab=False,
                tokenize=new_tokenizer, # 앞서 불러온 pre-trained BERT의 tokenizer를 가져와 사용한 후 앞 510개의 token만 골라 가져오는 함수로 수정해 대체
                preprocessing=PreProc, # data.Field의 preprocessing 옵션을 이용해 전처리 & Indexing
                init_token=tokenizer.cls_token_id,
                eos_token=tokenizer.sep_token_id,
                pad_token=tokenizer.pad_token_id,
                unk_token=tokenizer.unk_token_id)
LABEL=data.LabelField(dtype=torch.float)

In [35]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
LABEL.build_vocab(train_data)
train_data, valid_data = train_data.split(random_state = random.seed(0), split_ratio=0.8)

In [36]:
# Data Length
print(f'Train Data Length : {len(train_data.examples)}')
print(f'Test Data Length : {len(test_data.examples)}')

Train Data Length : 20000
Test Data Length : 25000


In [37]:
# Data Fields
train_data.fields

{'label': <torchtext.legacy.data.field.LabelField at 0x7fb98da6b690>,
 'text': <torchtext.legacy.data.field.Field at 0x7fb98e1e7790>}

In [38]:
# Data Sample
print('---- Data Sample ----')
print('Input : ')
print(tokenizer.convert_ids_to_tokens(vars(train_data.examples[2])['text']))

---- Data Sample ----
Input : 
['somewhere', 'in', 'his', 'non', '[UNK]', 'fiction', 'book', 'dans', '##e', 'mac', '##ab', '##re', '[UNK]', 'stephen', 'king', 'suggests', 'that', 'one', 'secret', 'of', 'writing', 'scary', 'stories', 'is', 'to', 'avoid', 'showing', 'your', 'readers', 'exactly', 'what', 'horrible', 'thing', 'is', 'waiting', 'behind', 'the', 'door', 'to', 'get', 'them', '[UNK]', 'if', 'at', 'last', 'the', 'door', 'bursts', 'open', 'and', 'a', 'bug', 'ten', 'feet', 'tall', 'lu', '##rch', '##es', 'through', '[UNK]', 'the', 'reader', 'may', 'be', 'a', 'little', 'scared', '[UNK]', 'but', 'he', "'", 'll', 'also', 'think', '[UNK]', '[UNK]', 'well', '[UNK]', 'i', 'can', 'deal', 'with', 'that', '[UNK]', 'at', 'least', 'it', 'wasn', "'", 't', 'a', 'hundred', 'feet', 'tall', '[UNK]', '[UNK]', 'there', "'", 's', 'nothing', 'more', 'frightening', 'than', 'what', 'lu', '##rks', '[UNK]', 'unseen', 'and', 'unknown', '[UNK]', 'just', 'on', 'the', 'other', 'side', 'of', 'that', 'tightly',

In [39]:
# Label Info
print(f'Label Size : {len(LABEL.vocab)}')

print('Lable Examples : ')
for idx, (k, v) in enumerate(LABEL.vocab.stoi.items()):
    print('\t', k, v)

Label Size : 2
Lable Examples : 
	 neg 0
	 pos 1


In [40]:
model_config = {}
model_config['batch_size'] = 10


train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=model_config['batch_size'])

In [41]:
bert = BertModel.from_pretrained('bert-base-uncased')
model_config['emb_dim'] = bert.config.to_dict()['hidden_size']
print(model_config['emb_dim'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768


In [42]:
class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(SentenceClassification, self).__init__()
        self.bert = bert
        self.fc = nn.Linear(model_config['emb_dim'],
                            model_config['output_dim'])
        
    def forward(self, x):
        pooled_cls_output = self.bert(x)[1] # output 자체를 쓰는 것이 아니라 pooled_output 사용 (BertModel 기준 두 번째 값 사용)
        return self.fc(pooled_cls_output)

In [44]:
model_config.update(dict(output_dim = 1))

model = SentenceClassification(**model_config)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5) # 이미 잘 학습된 BERT 모델 파라미터를 크게 변하게 되면 안되기 때문에 learning rate를 낮춰야 함.
loss_fn = nn.BCEWithLogitsLoss()

In [45]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model) 

109483009

In [46]:
def binary_accuracy(preds, y):
    # rounded_preds = torch.argmax(preds, axis=1) 
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum()/len(correct)
    return acc

In [ ]:
N_EPOCH = 4

best_valid_loss = float('inf')
model_name = "BERT"

print('---------------------------------')
print(f'Model name : {model_name}')
print('---------------------------------')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, loss_fn, epoch, **model_config)
    print('')
    print(f'\t Epoch : {epoch} | Train Loss : {train_loss:.4} | Train Acc : {train_acc:.4}')
    valid_loss, valid_acc = evaluate(model, valid_iterator, loss_fn, epoch, **model_config)
    print('')
    print(f'\t Epoch : {epoch} | Valid Loss : {valid_loss:.4} | Valid Acc : {valid_acc:.4}')
    # print('')
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f'./{model_name}.pt')
        print(f'\t Model is saved at {epoch}-epoch')

---------------------------------
Model name : BERT
---------------------------------


In [ ]:
# Test set
model.load_state_dict(torch.load(f'./{model_name}.pt'))
epoch = 0
test_loss, test_acc = evaluate(model, test_iterator, loss_fn, epoch, **model_config)
print('')
print(f'Test Loss : {test_loss:.4} | Test Acc : {test_acc:.4}')